# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4853, uptime 0:00:32
mini_beamline                    RUNNING   pid 4854, uptime 0:00:32
random_walk                      RUNNING   pid 4855, uptime 0:00:32
random_walk_horiz                RUNNING   pid 4856, uptime 0:00:32
random_walk_vert                 RUNNING   pid 4857, uptime 0:00:32
simple                           RUNNING   pid 4858, uptime 0:00:32
thermo_sim                       RUNNING   pid 4859, uptime 0:00:32
trigger_with_pc                  RUNNING   pid 4860, uptime 0:00:32


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-30 19:59:17
Persistent Unique Scan ID: '9af83b48-2628-4f11-99c4-4a3d6244ddb3'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 19:59:17.9 |     -1.000 |      97404 |


|         2 | 19:59:18.1 |     -0.500 |     100168 |
|         3 | 19:59:18.1 |      0.000 |     100816 |
|         4 | 19:59:18.1 |      0.500 |     100390 |
|         5 | 19:59:18.1 |      1.000 |      99195 |


+-----------+------------+------------+------------+
generator scan ['9af83b48'] (scan num: 3)





('9af83b48-2628-4f11-99c4-4a3d6244ddb3',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/9af83b48-2628-4f11-99c4-4a3d6244ddb3-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-30 19:59:18
Persistent Unique Scan ID: 'efb3715d-cefa-481f-8054-b8d4e73f611f'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 19:59:18.6 |          -1 | 1116755685 |


|         2 | 19:59:18.7 |          -0 | 1122773986 |


|         3 | 19:59:18.8 |           0 | 1129140038 |
|         4 | 19:59:18.9 |           0 | 1131064085 |


|         5 | 19:59:19.0 |           1 | 1131765737 |


+-----------+------------+-------------+------------+
generator scan ['efb3715d'] (scan num: 4)





('efb3715d-cefa-481f-8054-b8d4e73f611f',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-30 19:59:19
Persistent Unique Scan ID: '0ad498cc-e397-4361-82bc-09d05819a3c9'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 19:59:19.5 | 1125270121 |


|         2 | 19:59:20.4 | 1069312612 |


|         3 | 19:59:21.4 | 1061027417 |


...

|        28 | 19:59:46.5 | 1117591846 |


|        29 | 19:59:47.5 | 1122607787 |


|        30 | 19:59:48.5 | 1067092538 |


+-----------+------------+------------+
generator count ['0ad498cc'] (scan num: 5)





('0ad498cc-e397-4361-82bc-09d05819a3c9',)